算法抄襲自:
[github repo](https://github.com/lesterlitch/misc/blob/master/Light%2Bfm%2Bannoy%2Band%2Bproduct%2Bsearch%2Bexample.ipynb)

In [1]:
from scipy import sparse as sp
from lightfm import LightFM
from lightfm.evaluation import precision_at_k,recall_at_k,auc_score
import copy
import itertools
import numpy as np 
import lightfm
import pickle 
import pandas as pd 

C:\Program Files\Anaconda3\envs\py36\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [4]:
#fundid_names_df.to_csv('./funds/fundid_to_name.csv',index=False)

with open('./funds-dataset/sp_funds_datasets.pickle','rb') as f:
    data = pickle.load(f)
    
test = data['test']
train = data['train']
user_idxs = data['user_idxs']
idx_to_userid = data['idx_to_userid']
userid_to_idx = data['userid_to_idx']
idx_to_itemid = data['idx_to_itemid']
itemid_to_idx = data['itemid_to_idx']

fundid_names_df = pd.read_csv('./funds-dataset/fundid_to_name.csv',encoding='cp950')
fundid_to_names = {}

for d in fundid_names_df.to_dict('records'):
    fundid_to_names[d['基金代碼']] = d['基金中文名稱']
#%% 

In [5]:
model = LightFM(learning_rate=0.01, loss='warp')
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()
train_recall = recall_at_k(model,train,k=10).mean()
test_recall = recall_at_k(model,test,k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()
print('Recall: train {:.2f}%, test {:.2f}%'.format(100*train_recall,100*test_recall))
print('Precision: train {:.2f}% , test {:.2f}%.'.format(100*train_precision, 100*test_precision))
print('AUC: train {:.2f}, test {:.2f}.'.format(train_auc, test_auc))

Recall: train 21.57%, test 19.74%
Precision: train 9.64% , test 1.97%.
AUC: train 0.92, test 0.91.


In [7]:
def sample_recommendation_original(model, data, user_ids, print_output=True):
    
    train = data['train']
    test = data['test']
    assert isinstance(train,sp.csr_matrix) and isinstance(test,sp.csr_matrix)
        
    n_users, n_items = train.shape

    for user_id in user_ids:
        
        known_positives_itemids = [ 
                idx_to_itemid[e] for e in train[user_id].indices
                ]
        known_positives_item_names = [
                fundid_to_names[e] for e in known_positives_itemids
                ]
        scores = model.predict(user_id, np.arange(n_items))
        top_items_ids = [idx_to_itemid[e] for e in np.argsort(-scores)]
        if print_output == True:
            print("User %s" % user_id)
            print("     Known positives:")

            for x in known_positives_item_names[:3]:
                print("        %s" % x)

            print("     Recommended:")

            for x in top_items_ids[:3]:
                print("        %s" % fundid_to_names[x])

In [8]:
sample_recommendation_original(model,data,range(3))

User 0
     Known positives:
        (百元基金)摩根美國複合收益債券基金-JPM-A股累計(美元)
        富達中國聚焦基金(年配)-配息帳戶-美元
        (百元基金)貝萊德歐洲價值型基金HEDGED A2(美元)
     Recommended:
        (百元基金)貝萊德中國基金A2(美元)
        柏瑞印度股票基金A(美元)
        瑞銀(盧森堡)生化股票基金(美元)
User 1
     Known positives:
        柏瑞印度股票基金A(美元)
        (百元基金)貝萊德歐洲價值型基金HEDGED A2(美元)
        瑞銀(盧森堡)生化股票基金(美元)
     Recommended:
        安聯收益成長基金-AM(穩定月收類股)(美元)
        瑞銀(盧森堡)生化股票基金(美元)
        (百元基金)貝萊德中國基金A2(美元)
User 2
     Known positives:
        摩根日本(日圓)基金
        (百元基金)貝萊德世界能源基金(美元)
        (百元基金)貝萊德中國基金A2(美元)
     Recommended:
        (百元基金)貝萊德中國基金A2(美元)
        (百元基金)永豐滬深300紅利指數基金
        安聯收益成長基金-AM(穩定月收類股)(美元)


_____

Now we'll do an Annoy example showing how we can do user recommendations using a neat trick outlined by the 
[Xbox recomendations team]( https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/XboxInnerProduct.pdf)

In [13]:
def sample_recommendation_annoy(model,data,user_ids,print_output=True):
    train = data['train']
    test = data['test']
    n_users, n_items = train.shape

    for user_id in user_ids:
        
        known_positives_itemids = [ 
                idx_to_itemid[e] for e in train[user_id].indices
                ]
        known_positives_item_names = [
                fundid_to_names[e] for e in known_positives_itemids
                ]
        
        top_items_ids = [idx_to_itemid[e] for e in t_member.get_nns_by_vector(
            np.append(user_vectors[user_id], 0), 50)]
        
        if print_output == True:
            print("User %s" % user_id)
            print("     Known positives:")

            for x in known_positives_item_names[:3]:
                print("        %s" % x)

            print("     Recommended:")

            for x in top_items_ids[:3]:
                print("        %s" % fundid_to_names[x])

In [14]:
sample_recommendation_annoy(model,data,user_ids=[0,1,2])

User 0
     Known positives:
        (百元基金)摩根美國複合收益債券基金-JPM-A股累計(美元)
        富達中國聚焦基金(年配)-配息帳戶-美元
        (百元基金)貝萊德歐洲價值型基金HEDGED A2(美元)
     Recommended:
        (百元基金)摩根日本(日圓)基金-累積(美元對沖)
        柏瑞印度股票基金A(美元)
        (百元基金)富蘭克林坦伯頓全球生技領航基金(美元)
User 1
     Known positives:
        柏瑞印度股票基金A(美元)
        (百元基金)貝萊德歐洲價值型基金HEDGED A2(美元)
        瑞銀(盧森堡)生化股票基金(美元)
     Recommended:
        柏瑞印度股票基金A(美元)
        (百元基金)富蘭克林坦伯頓全球生技領航基金(美元)
        (百元基金)摩根日本(日圓)基金-累積(美元對沖)
User 2
     Known positives:
        摩根日本(日圓)基金
        (百元基金)貝萊德世界能源基金(美元)
        (百元基金)貝萊德中國基金A2(美元)
     Recommended:
        (百元基金)摩根日本(日圓)基金-累積(美元對沖)
        (百元基金)摩根印度基金
        鋒裕歐洲潛力基金A2(美元對沖)


In [9]:
import annoy 
from annoy import AnnoyIndex

In [10]:
item_vectors = model.item_embeddings

In [25]:
item_vectors.shape

(2149, 10)

In [28]:
np.sqrt(max_norm)

array([ 0.46629167,  1.59448087,  1.2161541 , ...,  0.44345409,
        0.38055053,  0.37847042], dtype=float32)

In [30]:
norm_data.shape

(2149, 11)

In [63]:
norms = np.linalg.norm(item_vectors, axis=-1)
# self.factors = artist_factors / norms[:, numpy.newaxis]

In [67]:
norms[:,np.newaxis]

array([[ 0.46629167],
       [ 1.59448087],
       [ 1.2161541 ],
       ..., 
       [ 0.44345409],
       [ 0.38055053],
       [ 0.37847042]], dtype=float32)

In [56]:
class ApproximateTopRelated(object):
    def __init__(self, artist_factors, treecount=20):
        index = annoy.AnnoyIndex(artist_factors.shape[1], 'angular')
        for i, row in enumerate(artist_factors):
            index.add_item(i, row)
        index.build(treecount)
        self.index = index

    def get_related(self, artistid, N=10):
        neighbours = self.index.get_nns_by_item(artistid, N)
        return sorted(((other, 1 - self.index.get_distance(artistid, other))
                      for other in neighbours), key=lambda x: -x[1])

In [57]:
approx_related = ApproximateTopRelated(item_vectors)
approx_related.get_related(10)

[(10, 1.0),
 (505, 0.3794142007827759),
 (157, 0.33101195096969604),
 (1138, 0.2974626421928406),
 (511, 0.2875285744667053),
 (1298, 0.2798996567726135),
 (412, 0.26625990867614746),
 (1031, 0.2592926025390625),
 (255, 0.24641066789627075),
 (1295, 0.2387155294418335)]

In [111]:
norms = np.linalg.norm(item_vectors, axis=1)
max_norm = norms.max()
extra_dimension = np.sqrt(max_norm ** 2 - norms ** 2)
norm_data = np.append(
    item_vectors, extra_dimension.reshape(norms.shape[0], 1), axis=1)


f_member = norm_data.shape[1]
t_member = AnnoyIndex(f_member,'angular')  # Length of item vector that will be indexed

for i in range(norm_data.shape[0]):
    v = norm_data[i]
    t_member.add_item(i, v)

t_member.build(200)

True

In [84]:
user_id = 10
topn = 10 
t_member.get_nns_by_vector(user_vectors[user_id],topn)

[17, 178, 55, 1, 810, 35, 363, 781, 5, 841]

recall, precision on annoy 

In [112]:
def eval_recommendation_test(model,test,topn=10,annoy=True):
    n_users,n_items = test.shape
    hr = 0
    n_test = 0
    for user_id in range(n_users):
        if test[user_id].indices:
            if annoy:
                top_items_idxs = t_member.get_nns_by_vector(np.append(user_vectors[user_id],0),topn)
            else:
                scores = model.predict(user_id, np.arange(n_items))
                top_items_idxs = np.argsort(-scores)[:topn]
                
            test_fundidx = test[user_id].indices
            hr += np.any(np.intersect1d(test_fundidx,top_items_idxs))
            n_test += 1
    return hr/n_test

In [115]:
%time rat10_annoy = eval_recommendation_test(model,test,annoy=True)
%time rat10 = eval_recommendation_test(model,test,annoy=False)

Wall time: 4.72 s
Wall time: 7.03 s


In [116]:
print('recall:{:.2f}% at topn=10, without approximation '.format(rat10*100))
print('recall:{:.2f}% at topn=10, with approximation '.format(rat10_annoy*100))

recall:19.74% at topn=10, without approximation 
recall:9.29% at topn=10, with approximation 


_____

In [ ]:
def nearest_items_Annoy(itemid, index, n=10, print_output=True):
    nn = index.get_nns_by_item(itemid, 10)
    if print_output == True:
        print('Closest to %s : \n' % movielens['item_labels'][movie_id])
    titles = [movielens['item_labels'][i] for i in nn]
    if print_output == True:
        print("\n".join(titles))



In [117]:
f = item_vectors.shape[1]  # Length of item vector that will be indexed
t = AnnoyIndex(f)
for i in range(item_vectors.shape[0]):
    v = item_vectors[i]
    t.add_item(i, v)

t.build(10)  # 10 trees

True

In [121]:
itemid = 0
print('fund:{}'.format(fundid_to_names[idx_to_itemid[itemid]]))
[fundid_to_names[idx_to_itemid[e]] for e in t.get_nns_by_item(0,10)]

fund:(百元基金)摩根美國複合收益債券基金-JPM-A股累計(美元)


['(百元基金)摩根美國複合收益債券基金-JPM-A股累計(美元)',
 '(百元基金)貝萊德新興歐洲基金(美元)',
 '富達歐洲平衡基金(累積)-配息帳戶-美元避險',
 '(百元基金)摩根多重收益基金-JPM-A股累計(美元對沖)',
 '第一金中國世紀基金(人民幣)',
 '富達全球入息基金(累積)-配息帳戶-美元',
 '(百元基金)摩根新興市場債券基金-JPM-A股累計(美元)',
 '(百元基金)富蘭克林坦伯頓全球新興國家固定收益基金(季配權)',
 '(百元基金)施羅德環球股息A基金(累積)(美元)',
 '亨德森遠見泛歐地產股票基金A2(美元避險)']